<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Packages" data-toc-modified-id="Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Packages</a></span></li><li><span><a href="#Merging-main-files" data-toc-modified-id="Merging-main-files-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Merging main files</a></span></li><li><span><a href="#Loading-primers" data-toc-modified-id="Loading-primers-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Loading primers</a></span></li></ul></div>

## Packages

In [3]:
%load_ext autoreload
%autoreload 2

from utils.loading import read_saved_file
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Merging main files

In [2]:
sequence_df = read_saved_file(type_to_read='Sequence')
taxonomy_df = read_saved_file(type_to_read='Taxonomy')

In [28]:
joined_df = sequence_df.merge(taxonomy_df.drop('file_num', axis=1), on = ['reference'], how='left')

In [35]:
working_df = joined_df.dropna(axis=0, subset=['kingdom'])

In [39]:
working_df.phylum.value_counts()

PROTEOBACTERIA    23510
FIRMICUTES        22212
BACTEROIDETES      8356
ACTINOBACTERIA     5315
CHLOROFLEXI        2530
                  ...  
NANOARCHAEOTA         4
DICTYOGLOMI           3
CD12                  3
GOUTA4                3
WS6                   2
Name: phylum, Length: 68, dtype: int64

## Loading primers

In [12]:
def load_primers(start_or_end: str = '') -> pd.DataFrame:
    
    seq_file = open('../data/{}_primers.fasta'.format(start_or_end))
    line_to_read = True
    list_of_reads = []
    while line_to_read:
        hvr = seq_file.readline()[1:-1]
        seq = seq_file.readline()[:-2]
        if hvr == '':
            line_to_read = False
        else:
            list_of_reads.append([hvr, seq, start_or_end])
    seq_file.close()
    return pd.DataFrame(list_of_reads, columns=['hvr', 'sequence', 'start_or_end'])

In [16]:
start_primers = load_primers('start')
end_primers = load_primers('end')

In [17]:
start_primers

,hvr,sequence,start_or_end
0,V2,AGYGGCGNACGGGTGAGTA,start
1,V3,CCTACGGGAGGCAGCA,start
2,V4,AYTGGGYDTAAAGN,start
3,V5,AGGATTAGATACCC,start
4,V6,TCGATGCAACGCGAAGA,start
5,V7,GYAACGAGCGCAACC,start
6,V8,ATGGCTGTCGTCAGC,start
7,V23,AGYGGCGNACGGGTGAGTA,start
8,V34,CCTACGGGRSGCAGCA,start
9,V35,CCTACGGGAGGCAGCA,start
